# Notes

This assignment is devoted to `pandas`. It covers indexing and filtering, and some `groupby` and `join` operations. The assignment roughly corresponds to Week 4 and the beginning of Week 5 of the course.

The main dataset you'll be using is [Titanic](https://www.kaggle.com/c/titanic). Please, note, that you must not rely on any specific location for the dataset, hence, any code like

```python
titanic_train = pd.read_csv("<location>/train.csv")
```

will fail and your notebook won't be validated and graded. Inputs to the functions are described explicitly in each case, and that's the only thing you can rely on.

In [ ]:
%pylab inline
plt.style.use("bmh")

In [ ]:
plt.rcParams["figure.figsize"] = (6,6)

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
STUDENT = "Alon Cohen"
ASSIGNMENT = 4
TEST = False

In [ ]:
if TEST:
    import solutions
    total_grade = 0
    MAX_POINTS = 16

# Indexing and filtering

### 1. Fixing age (1 point).

There are several known mistakes in the Titanic dataset.

Namely, [Julia Florence Siegel](https://www.encyclopedia-titanica.org/titanic-survivor/julia-florence-cavendish.html) (Mrs. Tyrell William Cavendish) is mistakenly marked as being 76 years old (the age she actually died, but many years after Titanic).

You must **replace the corresponding age value in the dataframe with her actual age at the time** (25) and return the dataset. Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. You must return a copy of the input dataframe, and not perform replacement in the original dataframe. Structure and indexing must be the same as in input.

In [ ]:
def fix_age(df):
    
    """Fix age for Julia Florence Siegel."""
    
    result_df = df.copy()
    result_df.loc[result_df.Name.str.contains('Julia Florence Siegel'),'Age'] = 25.0
    
    return result_df

In [ ]:
PROBLEM_ID = 1

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, fix_age)

### 2. Embarkment port distribution (1 point).

You must find the value counts for embarkment port (`Embarked` column) for the passengers, who travelled in 3-d class, were male and between 20 and 30 years old (both inclusive). No need to treat missing values separately.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. You must return **series**, indexed with values from `Embarked`, according to `.value_counts()` method semantics:

```
S    <number of passengers in 3-d class, embarked at S, 20<=Age<=30>
C    <...>
Q    <...>
Name: Embarked, dtype: int64
```

In [ ]:
def embarked_stats(df):
    
    """Calculate embarkment port statistics."""
    
    results_df = df.copy()
    results_df = results_df[(results_df['Pclass']==3) & (results_df['Sex']=='male') &  (results_df['Age'].between(20,30))]
        
    return results_df['Embarked'].value_counts()

In [ ]:
PROBLEM_ID = 2

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, embarked_stats)

### 3. Fill missing age values (1 point).

Some age values are missing in the Titanic dataset. You need to calculate average age over all passengers, and fill missing age values in `Age` column.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be a **new** dataframe with the same structure, but without missing values in `Age` column.

In [ ]:
def fix_age(df):
    
    """Fix missing age values."""
    
    results_df = df.copy()
    mean_age = results_df['Age'].mean()
    
    results_df['Age'] = results_df['Age'].fillna(mean_age)
    
    
    return results_df

In [ ]:
PROBLEM_ID = 3

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, fix_age)

### 4. Child travelling alone (1 point).

You must find a child (`Age<10`) on-board, who was travelling without siblings or parents and find a name of her nursemaid.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be a **tuple** of two strings, collected from `Name` column, with one being child's name and second being nursemaid's name. It's known, that there's **only one child** like this.

In [ ]:
def get_nursemaid(df):
    
    results_df = df.copy()
    child_info = results_df.loc[(results_df.Age<10) & (results_df.Parch==0)]
    child_id, ticket_id = child_info.index.item(), child_info.Ticket.item()
    nursemaid = results_df.loc[(results_df.index!=child_id) & (results_df.Ticket==ticket_id), 'Name'].item()

    return nursemaid

In [ ]:
PROBLEM_ID = 4

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, get_nursemaid)

### 5. Port with the most children embarked (1 point).

You must find, which port had the largest percentage of children (`Age<10`) embarked, i.e. number of children divided by total number of passengers embarked.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be a **single string** with port letter.

In [ ]:
def get_port(df):
    
    """Get port with the most children embarked."""
    results_df = df.copy()
    results_df = results_df.groupby([results_df.Age.apply(lambda x: x < 10),'Embarked']).agg(c = ('Name','count')).unstack(0).droplevel(0,axis=1)
    results_df = results_df.apply(lambda x: x[True] / (x[True] + x[False]),axis=1)
    
    return results_df.idxmax()


In [ ]:
PROBLEM_ID = 5

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, get_port)

### 6. Passengers per ticket (2 points).

Calculate average and maximum number of passengers per ticket.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be a **tuple** of two values - average and maximum number of passengers per ticket.

In [ ]:
def get_ticket_stats(df):

    """Calculate passenger per ticket statistics."""
    result_df = df.groupby('Ticket').agg({'Name':'count'}).Name
    
    return result_df.mean(),result_df.max()

In [ ]:
PROBLEM_ID = 6

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, get_ticket_stats)

### 7. Fare per passenger (3 points).

For each individual ticket, you must calculate **fare per person for that ticket**, and then calculate averages for each class. Note, that you will need to apply `groupby` and you may consider using `.first()` of resulting `DataFrameGroupBy`. Also, caferully consider, in which order calculations are performed.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be `pd.Series` with three elements, indexed by class:

```
1    <average per person fare in class 1>
2    <...>
3    <...>
Name: Pclass, dtype: float64
```

In [ ]:
def get_fare_per_pass(df):
    
    """Calculate fare per passenger for different classes."""
    result_df = df.groupby(['Ticket']).agg({'Name':'count','Fare':'first','Pclass':'first'},as_index=False)
    result_df['avg_fare_per_ticket'] = result_df.apply(lambda x: x.Fare / x.Name,axis=1)
    return result_df.groupby('Pclass').agg(avg = ('avg_fare_per_ticket','mean'))

In [ ]:
PROBLEM_ID = 7

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, get_fare_per_pass)

### 8. Fill missing age values (3 points).

In problem 3 you filled missing age values with global average over all passengers. Now, you need to fill them **according to class and sex**. For example, for a female passenger from 2d class, missing age value must be filled with average age of females in 2d class.

In this problem, you may need joins and `.apply()`, although there are several ways to get the same result.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be a **new** dataframe with the same structure as input, but without missing values in `Age` column.

In [ ]:
def fix_age_groupped(df):
    """Fill missing age values."""
    
    df_result = df.copy()
    by_sex_and_age = df.groupby(['Pclass','Sex']).agg({'Age':'mean'}).unstack(0).droplevel(0,axis=1)
    df_result['Age'] =  df.apply(lambda x: by_sex_and_age.loc[x.Sex,x.Pclass] if pd.isna(x.Age) else x.Age,axis=1)

    return df_result

In [ ]:
PROBLEM_ID = 8

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, fix_age_groupped)

### 9. Finding couples (3 points).

Based on the code from Lecture 5, build a dataframe of couples. Filter it by survival status: select those couples, in which only one of spouses survived or none of two. Built survival statistics by class, i.e. ratio of the number couples with partial survival or couples which died together, divided by total number of couples in class. If the survival status of one or both of spouses is not known, it must be considered as `0`.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be `Series` with three elements indexed by values from `Pclass` column (see P7 as a reference).

In [ ]:
def find_couples(df):
    """Fill missing age values."""

    results_df = df.copy()
    results_df['Survived'] = results_df['Survived'].fillna(0)
    results_df = results_df.replace(re.compile(r'\s+\(.*\)'), '')
    family_names = (results_df
                .replace(re.compile(r'\s+\(.*\)'), '')
                .replace(re.compile("Mrs."), "Mr."))[["Name", "Sex","Survived"]]
    family_names = family_names[(family_names.Sex=="female") & family_names.Name.str.contains("Mr.")]
    couples = pd.merge(left = results_df, right=family_names,left_on = 'Name', right_on = 'Name',suffixes=['_m','_f'])
    couples['total_survived'] = (couples.Survived_f + couples.Survived_m).astype(int)
    survival_matrix = couples.groupby(['Pclass','total_survived']).agg(n = ('Name','count')).unstack().droplevel(0,axis=1)
    return survival_matrix[[0,1]].sum(axis=1) / survival_matrix.sum(axis=1)

In [ ]:
PROBLEM_ID = 9

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, find_couples)

In [ ]:
if TEST:
    print(f"{STUDENT}: {int(100 * total_grade / MAX_POINTS)}")